# 01_load_adam_xpt_files
Purpose: Load data & make some simple Plots

This notebook shows how to:
1) Load ADaM `.xpt` files
2) Briefly describe key datasets
3) Make a couple of simple, sanity-check plots

**Provenance**  
Sample data: PHUSE CDISC pilot (`cdiscpilot01`)  
- ADaM folder: https://github.com/phuse-org/phuse-scripts/tree/master/data/adam/cdiscpilot01  
  - Or the updated 2018 version: https://github.com/phuse-org/phuse-scripts/blob/master/data/adam/cdiscpilot_update1.zip
- Download single folder (macOS): `brew install svn` then  
  `svn export https://github.com/phuse-org/phuse-scripts/trunk/data/adam/cdiscpilot01`  
- Download single folder (Windows via TortoiseSVN): “SVN Checkout…” the same URL above into your target directory.


## Datasets in this demo (high-level)

| File          | What it is (typical content) |
|---|---|
| `adsl.xpt`    | Subject-level analysis dataset (one row per subject; demographics, treatment assignments, flags). |
| `adae.xpt`    | Adverse events analysis dataset (analysis-ready AE terms, timing, severity/seriousness flags, relationships). |
| `advs.xpt`    | Vital signs (analysis-ready VS measures like BP, pulse, temp; PARAM/PARAMCD, visit/time variables). |
| `adtte.xpt`   | Time-to-event analysis dataset (start/stop, event/censor flags, analysis time). |
| `adlbc.xpt`   | Clinical chemistry labs (ALT, AST, ALP, BILI, etc.; analysis variables, baseline/shift flags). |
| `adlbh.xpt`   | Hematology labs (HGB, HCT, PLT, etc.). |
| `adlbhy.xpt`  | Hy’s Law lab derivations (bilirubin/ALT/AST combinations). |
| `adlbcpv.xpt` | Chemistry lab **parameter-value** layout (one result per row with PARAMCD). |
| `adlbhpv.xpt` | Hematology lab **parameter-value** layout. |
| `adadas.xpt`  | ADAS-Cog questionnaire analysis dataset. |
| `adcibc.xpt`  | CIBIC (Clinician’s Interview-Based Impression of Change) analysis dataset. |
| `adnpix.xpt`  | NPI-X (Neuropsychiatric Inventory) analysis dataset. |
| `define.xml`  | Define-XML metadata describing structures, variables, derivations, and value-level metadata. |
| `define2-0-0.xsl` | Stylesheet to render Define-XML in a browser. |

Descriptions reflect the Define-XML for this package. :contentReference[oaicite:0]{index=0}


# Imports

In [ ]:
# Minimal imports; pyreadstat is fast and preserves labels
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

import pyreadstat


In [ ]:
DATA_DIR = Path("../data/raw/").resolve()

def read_xpt(path: Path) -> pd.DataFrame:
    """
    Read a SAS XPT (transport) file into a pandas DataFrame with pyreadstat.
    Returns a DataFrame; value/variable labels are preserved in metadata if needed.
    """
    df, meta = pyreadstat.read_xport(str(path))
    df.attrs["meta"] = meta
    return df

def peek(df: pd.DataFrame, n: int = 5):
    """Quick peek: shape + head."""
    print(f"Shape: {df.shape}")
    display(df.head(n))


In [ ]:
# Choose a few commonly used domains for quick checks
files_to_load = {
    "ADSL": DATA_DIR / "adsl.xpt",
    "ADAE": DATA_DIR / "adae.xpt",
    "ADVS": DATA_DIR / "advs.xpt",
    "ADTTE": DATA_DIR / "adtte.xpt",
    "ADLBH": DATA_DIR / "adlbh.xpt",

    
}

# What each file stands for
domain_descriptions = {
    "ADSL": "Subject-Level Analysis Dataset",
    "ADAE": "Adverse Events Analysis Dataset",
    "ADVS": "Vital Signs Analysis Dataset",
    "ADTTE": "Time-to-Event Analysis Dataset",
    "ADLBH": "Hematology Labs"
}

loaded = {}
for name, path in files_to_load.items():
    if path.exists():
        desc = domain_descriptions.get(name, "")
        header = f"{name} — {desc}" if desc else name
        print("=" * len(header))
        print(header)
        print("=" * len(header))
        print(f"File: {path.name}\n")
        loaded[name] = read_xpt(path)
        peek(loaded[name], 3)
        print()  # spacer
    else:
        print(f"WARNING: {path.name} not found in {DATA_DIR}")


In [ ]:

# 1) Subjects per treatment arm (from ADSL)
if "ADSL" in loaded:
    adsl = loaded["ADSL"]
    arm_col = "ARM" if "ARM" in adsl.columns else ("TRT01P" if "TRT01P" in adsl.columns else None)
    if arm_col:
        counts = adsl[arm_col].value_counts(dropna=False).sort_index()
        plt.figure()
        counts.plot(kind="bar")
        plt.title(f"Subjects per {arm_col}")
        plt.xlabel(arm_col)
        plt.ylabel("N")
        plt.tight_layout()
        plt.show()
    else:
        print("ADSL loaded, but no ARM/TRT01P column found for the simple bar plot.")

# 2) AE counts by treatment arm (merge ADAE with ADSL via USUBJID)
if "ADSL" in loaded and "ADAE" in loaded:
    adsl = loaded["ADSL"]
    adae = loaded["ADAE"]
    key = "USUBJID" if "USUBJID" in adae.columns and "USUBJID" in adsl.columns else None
    arm_col = None
    for c in ["ARM", "TRT01P", "TRT01A"]:
        if c in adsl.columns:
            arm_col = c
            break
    if key and arm_col:
        merged = adae[[key]].merge(adsl[[key, arm_col]], on=key, how="left")
        ae_counts = merged[arm_col].value_counts(dropna=False).sort_index()
        plt.figure()
        ae_counts.plot(kind="bar")
        plt.title(f"Adverse Event records per {arm_col}")
        plt.xlabel(arm_col)
        plt.ylabel("AE records")
        plt.tight_layout()
        plt.show()
    else:
        print("Could not plot AE counts by arm (missing USUBJID or treatment variable).")


In [ ]:
# Pick a subject (change this to any USUBJID you want)
USUBJID = loaded["ADSL"]["USUBJID"].iloc[0]


print(f"Selected subject: {USUBJID}")

# Ensure lab datasets are available (chemistry first; pivot layout as fallback)
for fn in ["adlbc.xpt", "adlbcpv.xpt"]:
    p = DATA_DIR / fn
    if p.exists():
        nm = "ADLBC" if fn.startswith("adlbc.") else "ADLBCPV"
        if nm not in loaded:
            print(f"Loading {nm} from {fn} ...")
            loaded[nm] = read_xpt(p)


In [ ]:
def best_time_col(df: pd.DataFrame):
    """Return a tuple (colname, series) choosing a good time axis."""
    for col in ["ADTM", "ADT", "ADY", "ATPTN", "AVISITN"]:
        if col in df.columns:
            s = df[col].copy()
            if col in ["ADTM", "ADT"]:
                s = pd.to_datetime(s, errors="coerce")
            return col, s
    # Fallback: index
    return None, pd.Series(range(len(df)), index=df.index)

def plot_patient_vitals(advs: pd.DataFrame, usubjid: str):
    # Common PARAMCD candidates
    param_map = {
        "HR": ["HR", "PULSE"],
        "SYSBP": ["SYSBP", "SBP"],
        "DIABP": ["DIABP", "DBP"],
    }
    df = advs[advs["USUBJID"] == usubjid].copy()
    if df.empty:
        print("No ADVS records for this subject.")
        return

    # Normalize for matching (case-insensitive)
    paramcd = df.get("PARAMCD")
    if paramcd is None:
        print("ADVS has no PARAMCD; cannot identify vitals.")
        return
    paramcd = paramcd.astype(str).str.upper()
    df = df.assign(_PARAMCD_UP=paramcd)

    # Build a filtered long df for the params we want
    wanted = []
    for key, candidates in param_map.items():
        mask = df["_PARAMCD_UP"].isin([c.upper() for c in candidates])
        if mask.any():
            tmp = df[mask].copy()
            tmp["_PARAM_KEY"] = key  # normalize key name for legend order
            wanted.append(tmp)
    if not wanted:
        print("No HR/SBP/DBP-like parameters found in ADVS for this subject.")
        return
    dfv = pd.concat(wanted, ignore_index=True)

    # Choose time axis
    tcol, tseries = best_time_col(dfv)
    dfv = dfv.assign(_T=tseries, _AVAL=pd.to_numeric(dfv.get("AVAL"), errors="coerce"))
    dfv = dfv.sort_values("_T")

    # Assemble legend labels with units when unique
    legends = {}
    for key, sub in dfv.groupby("_PARAM_KEY"):
        units = sub.get("AVALU")
        unit_str = ""
        if units is not None:
            uniq = units.dropna().unique()
            unit_str = f" ({uniq[0]})" if len(uniq) == 1 else ""
        # Prefer PARAM label if present
        label_series = sub.get("PARAM")
        base = key if label_series is None else label_series.dropna().iloc[0] if not label_series.dropna().empty else key
        legends[key] = f"{base}{unit_str}"

    # Plot
    plt.figure()
    for key, sub in dfv.groupby("_PARAM_KEY"):
        plt.plot(sub["_T"], sub["_AVAL"], marker="o", linestyle="-", label=legends.get(key, key))
    plt.title(f"Vitals for {usubjid}")
    plt.xlabel(tcol or "index")
    plt.ylabel("Value")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:

def plot_patient_glucose(adlbc: pd.DataFrame, usubjid: str):
    # Try PARAMCD/TEST for glucose-like rows
    df = adlbc[adlbc["USUBJID"] == usubjid].copy()
    if df.empty:
        print("No lab records for this subject in the provided lab dataset.")
        return

    # Create uppercase helpers
    up_cols = {c: c for c in ["PARAMCD", "PARAM", "LBTESTCD", "LBTEST"] if c in df.columns}
    for c in up_cols:
        df[f"_{c}_UP"] = df[c].astype(str).str.upper()

    # Match glucose by common tokens
    glucose_mask = False
    if "_PARAMCD_UP" in df:
        glucose_mask = glucose_mask | df["_PARAMCD_UP"].str.contains("GLUC|GLU", regex=True)
    if "_PARAM_UP" in df:
        glucose_mask = glucose_mask | df["_PARAM_UP"].str.contains("GLUCOSE", regex=False)
    if "_LBTESTCD_UP" in df:
        glucose_mask = glucose_mask | df["_LBTESTCD_UP"].str.contains("GLUC|GLU", regex=True)
    if "_LBTEST_UP" in df:
        glucose_mask = glucose_mask | df["_LBTEST_UP"].str.contains("GLUCOSE", regex=False)

    g = df[glucose_mask].copy()
    if g.empty:
        print("No glucose-like records found for this subject.")
        return

    # Choose time axis
    tcol, tseries = best_time_col(g)
    g = g.assign(_T=tseries, _AVAL=pd.to_numeric(g.get("AVAL"), errors="coerce")).sort_values("_T")

    # Units for label if unique
    units = g.get("AVALU")
    unit_str = ""
    if units is not None:
        uniq = units.dropna().unique()
        unit_str = f" ({uniq[0]})" if len(uniq) == 1 else ""

    plt.figure()
    plt.plot(g["_T"], g["_AVAL"], marker="o", linestyle="-")
    plt.title(f"Glucose over time for {usubjid}")
    plt.xlabel(tcol or "index")
    plt.ylabel(f"Glucose{unit_str}" if unit_str else "Glucose")
    plt.tight_layout()
    plt.show()

# ---- Run the plots ----
# Vitals
if "ADVS" in loaded and not loaded["ADVS"].empty:
    plot_patient_vitals(loaded["ADVS"], USUBJID)
else:
    print("ADVS not loaded or empty; cannot plot vitals.")

# Labs: prefer ADLBC, fall back to ADLBCPV
lab_df = loaded.get("ADLBC", None)
if lab_df is None:
    lab_df = loaded.get("ADLBCPV", None)

if lab_df is not None and not lab_df.empty:
    plot_patient_glucose(lab_df, USUBJID)
else:
    print("No lab dataset (ADLBC/ADLBCPV) loaded or it is empty; cannot plot glucose.")
